In [ ]:
# PIMALUOS: Manhattan Land Use Optimization Demo

This notebook demonstrates the complete PIMALUOS workflow for Manhattan parcels.

In [ ]:
# Install if needed
# !pip install pimaluos

## 1. Load Manhattan Parcel Data

In [ ]:
from pimaluos.core import get_data_loader, ParcelGraphBuilder
import matplotlib.pyplot as plt

# Load Manhattan data
loader = get_data_loader("manhattan")
gdf, features = loader.load_and_compute_features()

print(f"Loaded {len(gdf)} parcels")
print(f"Feature dimensions: {features.shape}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 12))
gdf.plot(ax=ax, column='BuiltFAR', cmap='viridis', legend=True)
ax.set_title('Manhattan Built FAR')
plt.show()

## 2. Build Heterogeneous Graph

In [ ]:
# Build graph with 5 edge types
builder = ParcelGraphBuilder(
    gdf=gdf, 
    features=features,
    k_neighbors=10,
    edge_types=['spatial', 'visual', 'functional', 'infrastructure', 'regulatory']
)

graph = builder.build_hetero_data()

print(f"Node features shape: {graph['parcel'].x.shape}")
print(f"Edge types: {graph.edge_types}")
for et in graph.edge_types:
    print(f"  {et}: {graph[et].edge_index.shape[1]} edges")

## 3. Train Parcel GNN

In [ ]:
import torch
from pimaluos.models import ParcelGNN, train_epoch, evaluate

# Initialize model
model = ParcelGNN(
    input_dim=features.shape[1],
    hidden_dim=128,
    output_dim=64,
    edge_types=graph.edge_types,
    num_land_use_classes=5
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Training loop
losses = []
for epoch in range(50):
    loss = train_epoch(model, graph, optimizer)
    losses.append(loss)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: loss = {loss:.4f}")

# Plot training curve
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('GNN Training')
plt.show()

## 4. Multi-Agent Land Use Optimization

In [ ]:
from pimaluos.models import StakeholderAgent, MultiAgentEnvironment, ConsensusVotingMechanism
from pimaluos.physics import MultiPhysicsEngine

# Initialize physics
physics = MultiPhysicsEngine(gdf)

# Create environment
env = MultiAgentEnvironment(
    gdf=gdf,
    gnn_model=model,
    physics_engine=physics,
    num_parcels=min(100, len(gdf))  # Limit for demo
)

# Create stakeholder agents
agent_types = ['resident', 'developer', 'planner', 'environmentalist', 'equity_advocate']
agents = {
    t: StakeholderAgent(state_dim=128, action_dim=3, agent_type=t)
    for t in agent_types
}

# Consensus mechanism
consensus = ConsensusVotingMechanism(agents, strategy='weighted')

print(f"Created {len(agents)} stakeholder agents")

## 5. Run Optimization Episode

In [ ]:
# Run episode
state = env.reset()
episode_rewards = {t: [] for t in agent_types}

for step in range(10):
    # Get actions from each agent
    actions = {name: agent.select_action(state) for name, agent in agents.items()}
    
    # Get consensus action
    consensus_action = consensus.vote(actions)
    
    # Step environment
    next_state, rewards, done, info = env.step({'consensus': consensus_action})
    
    for t in agent_types:
        episode_rewards[t].append(rewards.get(t, 0))
    
    state = next_state
    
    if done:
        break

# Plot rewards
for t in agent_types:
    plt.plot(episode_rewards[t], label=t)
plt.xlabel('Step')
plt.ylabel('Reward')
plt.legend()
plt.title('Agent Rewards')
plt.show()

## 6. Physics Validation

In [ ]:
# Validate final configuration
results = info.get('physics_results', {})

print("=" * 40)
print("PHYSICS VALIDATION RESULTS")
print("=" * 40)

if results:
    print(f"Traffic congestion: {results['traffic']['avg_congestion_ratio']:.2f}")
    print(f"Drainage utilization: {results['hydrology']['capacity_utilization']:.1%}")
    print(f"Solar shadow: {results['solar']['avg_shadow_pct']:.1f}%")
    print(f"Total violations: {results['violations']['total_violations']}")
else:
    print("Run simulation to see results")

## 7. Visualize Results

In [ ]:
# Get attention weights for interpretability
from pimaluos.models import AttentionVisualizer

viz = AttentionVisualizer(model)

# Extract attention for a sample parcel
sample_idx = 0
attention_weights = viz.extract_attention_weights(graph)

print(f"Attention weights for parcel {sample_idx}:")
for edge_type, weights in attention_weights.items():
    print(f"  {edge_type}: mean={weights.mean():.4f}")